**Build and Train GRU model with Implementation of Hyperparameter Tuning**

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import KFold, train_test_split
import numpy as np
import pandas as pd

In [12]:
X = np.load('rolling_window_sequences.npy')  # Replace with actual file path
metadata_test = pd.read_csv("sequence_metadata_with_RUL.csv")  # Replace with actual file path
y = metadata_test["RUL"].values

print("Feature shape:", X.shape)
print("Target shape:", y.shape)


Feature shape: (17631, 30, 66)
Target shape: (17631,)


In [13]:
# OPTIONAL: Initial train-validation split (before CV)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
# Define hyperparameters to tune
params = {
    'units': [32, 64],
    'batch_size': [16, 32],
    'epochs': 50,
    'learning_rate': [0.001, 0.0005]
}

In [15]:
# Set up 5-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [16]:
# Define a function to build the GRU model
def build_gru_model(units, learning_rate, input_shape):
    model = Sequential()
    model.add(GRU(units, input_shape=input_shape, return_sequences=False))
    model.add(Dense(1))  # Single output for regression (RUL)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [17]:
# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_gru_model.keras', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-5, verbose=1)
]

In [22]:
best_val_loss = np.inf
best_model = None
best_params = None

# Hyperparameter tuning with cross-validation
for units in params['units']:
    for batch_size in params['batch_size']:
        for learning_rate in params['learning_rate']:
            val_losses = []
            for train_idx, val_idx in kfold.split(X):
                X_train, X_val = X[train_idx], X[val_idx]
                y_train, y_val = y[train_idx], y[val_idx]

                model = build_gru_model(units, learning_rate, input_shape=(X.shape[1], X.shape[2]))

                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=params['epochs'],
                    batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=0
                )
                val_loss = min(history.history['val_loss'])
                val_losses.append(val_loss)

            avg_val_loss = np.mean(val_losses)
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_model = model
                best_params = (units, batch_size, learning_rate)

In [23]:
print(f"Best validation loss: {best_val_loss}")
print(f"Best parameters: units={best_params[0]}, batch_size={best_params[1]}, learning_rate={best_params[2]}")

Best validation loss: 1857.0008544921875
Best parameters: units=32, batch_size=16, learning_rate=0.001


In [24]:
# Save the best model
best_model.save('final_best_gru_model_cmapss.keras')